In [1]:
from time import time
import numpy as np
from tst.domutil.util import *
from tst.domutil.test import *
from copy import copy
from scipy.sparse import *
from scipy.spatial.distance import *


def normalise(hc1, hc2, hcboth):
    log1 =  np.log10( hc1 + 1) 
    log2 =  np.log10( hc1 + 1) 
    log3 =  np.log10( hcboth + 1) 
    norm = (log1 + log2) / 2 - log3 
    return norm

reset_database_connection()

/export/people/zcbtfge/Documents/repos/test_envs/django_CATH/local/lib/python2.7/site-packages/subprocess32.py:472: RuntimeWarning: The _posixsubprocess module is not being used. Child process reliability may suffer if your program uses threads.
  "program uses threads.", RuntimeWarning)


In [2]:
# import sys
reload(sys.modules["tst.domutil.util"])
from tst.domutil.util import *

In [17]:
import sys
reload(sys.modules["tst.domutil.test"])
from tst.domutil.test import *

In [2]:
%%time
reset_database_connection()
# print "finished"
hmms = HMMprofile.objects.defer('text').all().prefetch_related('hits')
hmms = hmms.annotate(hcount = Count('hits')).order_by('id')

hmmids = list(hmms.values_list('id',flat = True))
hcounts = list(hmms.values_list('hcount',flat = True))
sfs = list(hmms.values_list('cath_node__parent',flat = True))

sDB = seqDB.objects.get(name = 'CATH')
# sDB.sequence_set.count()
seq = sDB.sequence_set.first()

seqset_all = sDB.sequence_set.all().prefetch_related('hmmprofile_set')

# seq.HMMprofile
# seq.hmmprofile
# dir(seq)
# seq.hmmprofile_set.count()


CPU times: user 84 ms, sys: 0 ns, total: 84 ms
Wall time: 50.3 s


In [ ]:
%%time
X0 = hmms2hit_ids(hmms)

In [8]:
%%time

# t0 = time()
# c = counter(range(len(X)**2/2),per = 1E3)
# from multiprocessing import *

hmms_test = HMMprofile.objects.defer('text').all().prefetch_related('hits')[:200]
# hmms_test = HMMprofile.objects.defer('text').all().prefetch_related('hits')[:500]
X = hmms2hit_ids( hmms_test )

CPU times: user 324 ms, sys: 12 ms, total: 336 ms
Wall time: 442 ms


In [ ]:
from tst.domutil.util import *
# its = D.iteritems()
its = using_tocoo_izip(D)

xs = []
ys = []
zs = []
sfbools = [];

c = counter(range(6563388),per = 100000)

for xi,yi,inter_ct in its:
    c.count()
#     xct = 
    x = hcounts[xi]
    y = hcounts[yi]

    xs.append( x )
    ys.append( y )
    zs.append(inter_ct)
    sfbools.append(sfs[xi]==sfs[yi])
#     normz = log10()
# print next(its)
# print next(its)

# ys 

In [31]:
from scipy.sparse import *
D = coo_matrix([])

In [28]:
seq = seqset_all[2]
field = 'cath_node__parent'
ids = seq.hmmprofile_set.values_list( field,flat = True)
%timeit ids.distinct()
%timeit set(ids)
%timeit ids
# %timeit set(seq.hmmprofile_set.values_list( field,flat = True)))
# %timeit len(seq.hmmprofile_set.values_list( field,flat = True))


100000 loops, best of 3: 9.53 µs per loop
The slowest run took 1283.46 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 555 ns per loop
100000000 loops, best of 3: 16.8 ns per loop


# Counting HMM-HMM intermediate sequence
## (sequence wise, Parallel)

In [23]:
sorted({1,2})

[1, 2]

In [3]:
#### Defining Parallel Wrappers ####

import itertools as it
from collections import Counter

# def pair_gen(seqset_all):
#     c = counter(range(seqset_all.count()))
#     batches = batch_qs( seqset_all, 500)
#     for seqset in batches:
        
# #             hmmids = seq.values_list("hmmprofile",flat =True)
#         for seq in seqset:
#             c.count()
#             hmmids = seq.hmmprofile_set.values_list("id",flat = True).distinct()
# #             pairs = it.combinations(sorted(hmmids),2)
#             pairs = it.combinations( hmmids ,2)
#             for p in pairs:
#                 yield p

def batch_worker(seqset,field="cath_node"):
    count = Counter()
    for seq in seqset:
        c0.count()
        ids = set( seq.hmmprofile_set.values_list( field,flat = True) )
    #             pairs = it.combinations(sorted(hmmids),2)
        pairs = it.combinations( sorted(ids) ,2)
        count.update(pairs)
    q.put(count)
#     c1.count()
    return 


def seq_counter(seq):
    c0.count()
    hmmids = seq.hmmprofile_set.values_list("id",flat = True)
#             pairs = it.combinations(sorted(hmmids),2)
    pairs = it.combinations( hmmids ,2)
    count = Counter(pairs)
    q.put(count)
    
    
def listener( OUTPUT ):
#     OUTPUT should be a dok matrix !!!!
#     D_curr = dok_matrix( OUTPUT )
    counts = Counter([])    
    i = 0

    while 1:
        print i
        i += 1
        obj = q.get()
        if obj:
            c1.count()
            counts.update(obj)            
        else:
            ################################################################
            ###### Put counts into a sparse matrix #########################
            ################################################################     
#             OUTPUT = OUTPUT.todok()]
#             counts
#             d = dict()
            for (x,y),v in counts.iteritems():
                OUTPUT.update({
                    (fdict[x] , fdict[y]) :v
                })
#             OUTPUT.update(counts)

#             q.put( OUTPUT.todok() ) ####### DOK matrix breaks if put on a Queue() for unknown reason
            q.put( OUTPUT.tocoo() )
#             q.put(counts)
            break
    return
    
def listener_mat( OUTPUT ):
#     from scipy.sparse import *
    D_curr = OUTPUT
    i = 0
    while 1:
        print i
        i += 1
        obj = q.get()
        if obj:
            c1.count()
            for k,v in obj.iteritems():
                D_curr[k] += v
        else:
            q.put( D_curr )
            return



In [6]:
%%time
seqset_all = sDB.sequence_set.all().prefetch_related('hmmprofile_set')


CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 355 µs


In [39]:
letter = 'H'

In [6]:
forward_field = {
    'S':'cath_node',
    'H':'cath_node__parent',
    'T':'cath_node__parent__parent',
    'A':'cath_node__parent__parent__parent',
    'C':'cath_node__parent__parent__parent__parent',
}

reverse_field = {
    'S':'hmmprofile__hits',
    'H':'classification__hmmprofile__hits',
    'T':'classification__classification__hmmprofile__hits',
    'A':'classification__classification__classification__hmmprofile__hits',
    'C':'classification__classification__classification__classification__hmmprofile__hits'
}

In [14]:
letter = 'H'
letter = 'S'

field = forward_field[letter]

reverse_dict = sorted(set( HMMprofile.objects.values_list( field,flat = True)))
forward_dict = list2dict(reverse_dict)


In [12]:
field

'cath_node__parent__id'

In [15]:
%%time

##### Actually firing subprocesses #####

from scipy.sparse import *
import multiprocessing as mp
from multiprocessing.managers import BaseManager, SyncManager
class MyManager(SyncManager): pass
MyManager.register('counter',counter)

reset_database_connection()
# c = counter(range(sDB.sequence_set.count()))

pcount = 5
# INPUT = seqset_all[:1000]
INPUT = seqset_all
ct1 = INPUT.count()
bsize = 600 
INPUTs = batch_qs( INPUT, bsize )

field = forward_field[letter]

reverse_dict = sorted(set( HMMprofile.objects.values_list(field,flat = True)))
forward_dict = list2dict(reverse_dict)

l = len(reverse_dict)


####Deprecated# OUTPUT = Counter()
OUTPUT = dok_matrix( (l + 1, l + 1), dtype = 'int')
# local_listener = listener_mat      
local_listener = listener      

local_worker = batch_worker
# local_worker = seq_counter

if __name__=='__main__':
    
    #     m = mp.Manager()
        m = MyManager()
        m.start()
    #     raise Exception
    #     manager.start()
        c0 = m.counter( range(ct1),INF=0, ifprint = 1,  prefix = '[Worker]',
                      per = bsize,)
        c1 = m.counter( [],INF=1, ifprint = 1, prefix = '[Listener]',
                      step = bsize, per = bsize, )
        
        fdict = m.dict(forward_dict)

        q = m.Queue();          

        if 1:
            pool = mp.Pool( pcount )
            watcher  = mp.Process(target = local_listener, args = [OUTPUT])
            watcher.start()
#             watcher  = pool.apply_async(local_listener, args = [OUTPUT])
            jobs = []

            for INPUT_curr in INPUTs:
                job  = pool.apply_async(local_worker, (INPUT_curr,), {'field':field})
                jobs.append(job)
            for job in jobs:
                job.get()

            q.put(None)
#             OUTPUT = watcher.get()
            watcher.join()
            OUTPUT = q.get() 
        pool.close()
        pool.join()

0
[Worker]0 of 1000
[Worker]600 of 1000
[Listener]0 of -1
1
[Listener]600 of -1
2
CPU times: user 3.29 s, sys: 276 ms, total: 3.57 s
Wall time: 39.7 s


In [18]:
test__raw( OUTPUT, reverse_dict = reverse_dict, letter = letter, seqDB_curr = sDB)


AssertionError: [ERROR] 2.60.40.10.340.0 against 2.60.40.10.435.0 overlaps: Expected: 3404, Actual: 65 from 

In [88]:
d = dict()
for (x,y),v in r.iteritems():
    d.update(
    {(dct_id[x] , dct_id[y]):v}
    )

CPU times: user 10min 23s, sys: 35.4 s, total: 10min 58s
Wall time: 14min 14s

In [220]:
s35_1.first()._meta.get_fields()

(<ManyToOneRel: tst.classification>,
 <OneToOneRel: tst.node_stat>,
 <OneToOneRel: tst.domain>,
 <ManyToOneRel: tst.sequence>,
 <OneToOneRel: tst.hmmprofile>,
 <ManyToOneRel: tst.hit4cath2cath>,
 <ManyToOneRel: tst.hit4cath2cath>,
 <django.db.models.fields.AutoField: id>,
 <django.db.models.fields.IntegerField: Class>,
 <django.db.models.fields.IntegerField: arch>,
 <django.db.models.fields.IntegerField: topo>,
 <django.db.models.fields.IntegerField: homsf>,
 <django.db.models.fields.IntegerField: s35>,
 <django.db.models.fields.IntegerField: s60>,
 <django.db.models.fields.IntegerField: s95>,
 <django.db.models.fields.IntegerField: s100>,
 <django.db.models.fields.related.ForeignKey: version>,
 <django.db.models.fields.related.ForeignKey: level>,
 <django.db.models.fields.related.ForeignKey: parent>)

In [7]:
# test__raw( OUTPUT, dct_id, letter = '')
D_raw = OUTPUT

[OK] HMM for 3.40.50.720.379.0  against HMM for 3.40.50.720.491.0  overlaps 3, with 3 from 
0 of -1
[OK] HMM for 3.40.30.10.192.0  against HMM for 3.40.30.10.217.0  overlaps 326, with 326 from 
[OK] HMM for 3.40.50.300.138.0  against HMM for 3.40.50.2300.249.0  overlaps 9, with 9 from 
[OK] HMM for 3.55.50.10.1.0  against HMM for 3.40.50.720.359.0  overlaps 4, with 4 from 
[OK] HMM for 3.20.20.210.2.0  against HMM for 3.20.20.140.5.0  overlaps 1, with 1 from 


In [33]:
sids = set(sDB.sequence_set.values_list('id'))

In [188]:
# from ..utils import *
forward_field = {
    'S':'cath_node__id',
    'H':'cath_node__parent__id',
    'T':'cath_node__parent__parent__id',
    'A':'cath_node__parent__parent__parent__id',
    'C':'cath_node__parent__parent__parent__parent__id',
}

reverse_field = {
    'S':'hmmprofile__hits',
    'H':'classification__hmmprofile__hits',
    'T':'classification__classification__hmmprofile__hits',
    'A':'classification__classification__classification__hmmprofile__hits',
    'C':'classification__classification__classification__classification__hmmprofile__hits'
}
def test__raw(D_raw, reverse_dict, letter = 'S', seqDB_curr = None,**kwargs):
    # next(counts.iteritems()[0]
    # it = counts.iteritems()
    it = using_tocoo_izip(D_raw)
    c = counter([],INF = 1)#
    
    sids = set(seqDB_curr.sequence_set.values_list('id'))
    rv_field = reverse_field[letter]
    nodes = classification.objects.filter(level__letter = letter)
    
    for x,y,v in it:
#         print x,y
        ###### New routine
        hmm1 = nodes.filter(id = reverse_dict[x])
        hmm2 = nodes.filter(id = reverse_dict[y])
        hmm1hits = hmm1.values_list( rv_field,flat = True)
        hmm2hits = hmm2.values_list( rv_field,flat = True)
        ###### Old routine
#         hmm1 = hmms.get(id = x + 1)
#         hmm2 = hmms.get(id = y + 1)
#         hmm1hits = hmm1.hits.values_list('id')
#         hmm2hits = hmm2.hits.values_list('id')
        hmm1hits = set(hmm1hits) & sids
        hmm2hits = set(hmm2hits) & sids
        interhits = set(hmm1hits) & set(hmm2hits)
    
#         interhits = set(hmm1hits) & set(hmm2hits)
        exp = len(interhits)
        act = v
        
#         print v
#         print intercount
        msg = '%s against %s overlaps: Expected: %d, Actual: %d from '%(hmm1[0],hmm2[0], exp, act )
        print msg
        success = ( v == len(interhits) )
        errmsg = "[ERROR] " + msg
        assert success, errmsg
        print "[OK] " + msg
#         if success:
#             print "[OK] " + msg
#         else:
#             print "[ERROR] " + msg
        c.count()
        if c.i == 5:
            break
# if 
# test__raw( OUTPUT, reverse_dict = reverse_dict, letter = 'H', seqDB_curr = sDB)
def test__norm(D_curr, reverse_dict,  letter = 'S', seqDB_curr = None, dnum = 2,
               norm_func = ISS_normalise):
    it = using_tocoo_izip(D_curr)
    c = counter([],INF = 1)

    sids = set(seqDB_curr.sequence_set.values_list('id'))
    rv_field = reverse_field[letter]
    nodes = classification.objects.filter(level__letter = letter)
    
    for x,y,v in it:
        # hmm1 = hmms.get(id = x + 1)
        # hmm2 = hmms.get(id = y + 1)
        hmm1 = nodes.filter(id = reverse_dict[x])
        hmm2 = nodes.filter(id = reverse_dict[y])
        hmm1hits = hmm1.values_list( rv_field )
        hmm2hits = hmm2.values_list( rv_field )
        hmm1hits = set(hmm1hits) & sids
        hmm2hits = set(hmm2hits) & sids
        interhits = set(hmm1hits) & set(hmm2hits)
    
        intercount = len(interhits)
        
        exp = norm_func( len(hmm1hits), len(hmm2hits), intercount)
        act = v
#         print v
#         print intercount
        msg = '%s against %s overlaps: Expected: %f, Actual: %f from '%(hmm1[0],hmm2[0], exp, act )
#         print msg
        
        success = ( round(act, dnum) == round(exp, dnum) )
        errmsg = "[ERROR] " + msg
#         assert success, errmsg
#         print "[OK] " + msg
        if success:
            print "[OK] " + msg
        else:
            print  errmsg
        c.count()
        if c.i == 5:
            break

test__norm( OUTPUT, reverse_dict, letter, seqDB_curr = sDB,
          norm_func = ISS_normalise_new)

[OK] 1.10.150.240.0 against 4.10.1080.10.0 overlaps: Expected: 0.700187, Actual: 0.699858 from 
0 of -1
[OK] 1.10.730.10.0 against 3.40.50.300.0 overlaps: Expected: 0.373085, Actual: 0.372815 from 
[OK] 3.40.50.1010.0 against 3.40.50.1240.0 overlaps: Expected: 0.282451, Actual: 0.282023 from 
[OK] 1.10.10.10.0 against 2.10.230.10.0 overlaps: Expected: 0.113784, Actual: 0.113780 from 
[OK] 2.60.200.20.0 against 3.20.70.20.0 overlaps: Expected: 0.171979, Actual: 0.171157 from 


In [11]:
test__raw( OUTPUT, reverse_dict, letter = letter, seqDB_curr = sDB, debug = 1,depth = 100)


[ERROR] 3.40.50.1000.0 against 3.40.50.2000.0 overlaps: Expected: 0, Actual: 2 from 
0 of -1
[ERROR] 1.10.490.50.0 against 3.90.20.10.0 overlaps: Expected: 0, Actual: 1 from 
[ERROR] 1.10.269.10.0 against 3.90.148.10.0 overlaps: Expected: 0, Actual: 1 from 
[ERROR] 1.10.150.240.0 against 4.10.1080.10.0 overlaps: Expected: 0, Actual: 4 from 
[ERROR] 1.10.285.10.0 against 3.40.50.20.0 overlaps: Expected: 0, Actual: 1 from 
[ERROR] 2.60.260.40.0 against 3.30.40.10.0 overlaps: Expected: 0, Actual: 5 from 
[ERROR] 3.40.190.10.0 against 3.90.20.10.0 overlaps: Expected: 0, Actual: 1 from 
[ERROR] 1.10.1070.11.0 against 3.30.420.40.0 overlaps: Expected: 0, Actual: 2 from 
[ERROR] 3.20.20.100.0 against 3.20.20.120.0 overlaps: Expected: 0, Actual: 1 from 
[ERROR] 1.20.58.70.0 against 1.20.250.10.0 overlaps: Expected: 0, Actual: 1 from 
[ERROR] 1.10.1420.10.0 against 3.30.910.20.0 overlaps: Expected: 0, Actual: 1 from 
[ERROR] 1.10.1330.10.0 against 1.10.4030.10.0 overlaps: Expected: 0, Actual: 2

[ERROR] 2.10.70.10.0 against 2.30.30.40.0 overlaps: Expected: 0, Actual: 1 from 


In [47]:
# test__raw( OUTPUT, seqDB_curr = sDB, **locals())


TypeError: test__raw() got an unexpected keyword argument 'chebyshev'

In [42]:
%%time
#### Cache the raw ISS matrix

import cPickle as pk

test__raw( OUTPUT, reverse_dict, letter, seqDB_curr = sDB)
# test__raw( D_raw,hmms)
test__key( OUTPUT )
D_raw = OUTPUT

fname = 'data/%s_D_raw' % ( letter )
pk.dump(D_raw.tocoo(),open(  fname ,'wb'))

# if 'D_raw' not in locals().keys():
#     D_raw = pk.load(open(fname, 'rb'))

    
# test__raw(D_raw,hmms)


<QuerySet [<classification: 1.10.150.240.0>]> against <QuerySet [<classification: 4.10.1080.10.0>]> overlaps: Expected: 27, Actual: 27 from 
[OK] <QuerySet [<classification: 1.10.150.240.0>]> against <QuerySet [<classification: 4.10.1080.10.0>]> overlaps: Expected: 27, Actual: 27 from 
0 of -1
<QuerySet [<classification: 1.10.730.10.0>]> against <QuerySet [<classification: 3.40.50.300.0>]> overlaps: Expected: 10, Actual: 10 from 
[OK] <QuerySet [<classification: 1.10.730.10.0>]> against <QuerySet [<classification: 3.40.50.300.0>]> overlaps: Expected: 10, Actual: 10 from 
<QuerySet [<classification: 3.40.50.1010.0>]> against <QuerySet [<classification: 3.40.50.1240.0>]> overlaps: Expected: 3, Actual: 3 from 
[OK] <QuerySet [<classification: 3.40.50.1010.0>]> against <QuerySet [<classification: 3.40.50.1240.0>]> overlaps: Expected: 3, Actual: 3 from 
<QuerySet [<classification: 1.10.10.10.0>]> against <QuerySet [<classification: 2.10.230.10.0>]> overlaps: Expected: 1, Actual: 1 from 
[OK

In [6]:
# fname = 'data/D_raw'
D_raw = pk_load('D_raw')

test__raw(D_raw,hmms)


[OK] HMM for 2.60.40.10.648.0  against HMM for 2.60.40.10.369.0  overlaps 255, with 255 from 
0 of -1
[OK] HMM for 3.40.50.1820.108.0  against HMM for 3.40.50.1820.70.0  overlaps 761, with 761 from 
[OK] HMM for 3.40.50.300.171.0  against HMM for 3.40.50.2300.51.0  overlaps 10, with 10 from 
[OK] HMM for 2.60.40.10.290.0  against HMM for 2.10.70.10.35.0  overlaps 2, with 2 from 
[OK] HMM for 2.130.10.10.46.0  against HMM for 2.120.10.10.8.0  overlaps 86, with 86 from 


In [56]:
%%time
########################################
### Old routine for letter = 'S' #######
########################################
########################################

# norm_count = dict()
OUTPUT = dict()

l = len(reverse_dict)
OUTPUT = dok_matrix( (l+1,l+1), dtype =  np.float)
# INPUT = D_raw.todok().iteritems()
INPUT = D_raw
it  = using_tocoo_izip( INPUT )

# c=counter(range(6577065),per=100000)
c=counter(it,per=1000)
it  = using_tocoo_izip( INPUT )

h1s = []
h2s = []
h3s = []
for x,y,v in it:
# for (x,y),v in it:
    
    c.count()
    h1s += [hcounts[x-1]]
    h2s += [hcounts[y-1]]
    h3s += [v]

BUFlst = ISS_normalise_new(
    np.array(h1s),
    np.array(h2s),
    np.array(h3s),
)



it  = using_tocoo_izip( INPUT )
d = dict()
for (x,y,_),v in izip( it, BUFlst):
    k = (x,y)
    d[k] = v

# norm_count = OUTPUT
# for x,y,v in using_tocoo_izip(D_raw):
# D_norm = copy(D_raw.todok())

# l = max(hmmids) + 1
# l = len(reverse_dict)
# mat = dok_matrix( shape = (l+1,l+1), dtype =  np.float)
# mat.update(OUTPUT)
OUTPUT.update(d)

# D_norm = dok_matrix( (l+1,l+1),np.float)
# D_norm.update( OUTPUT )



0 of 58969
1000 of 58969
2000 of 58969
3000 of 58969
4000 of 58969
5000 of 58969
6000 of 58969
7000 of 58969
8000 of 58969
9000 of 58969
10000 of 58969
11000 of 58969
12000 of 58969
13000 of 58969
14000 of 58969
15000 of 58969
16000 of 58969
17000 of 58969
18000 of 58969
19000 of 58969
20000 of 58969
21000 of 58969
22000 of 58969
23000 of 58969
24000 of 58969
25000 of 58969
26000 of 58969
27000 of 58969
28000 of 58969
29000 of 58969
30000 of 58969
31000 of 58969
32000 of 58969
33000 of 58969
34000 of 58969
35000 of 58969
36000 of 58969
37000 of 58969
38000 of 58969
39000 of 58969
40000 of 58969
41000 of 58969
42000 of 58969
43000 of 58969
44000 of 58969
45000 of 58969
46000 of 58969
47000 of 58969
48000 of 58969
49000 of 58969
50000 of 58969
51000 of 58969
52000 of 58969
53000 of 58969
54000 of 58969
55000 of 58969
56000 of 58969
57000 of 58969
58000 of 58969
CPU times: user 372 ms, sys: 16 ms, total: 388 ms
Wall time: 357 ms


In [124]:
all_nodes = classification.objects.filter(version__name='v4_1_0')
qset = all_nodes.filter(level__letter = letter).annotate(hcount=Count(rv_field))
hcounts =  dict( qset.values_list('id','hcount') )

In [ ]:
sids = set(sDB.sequence_set.values_list('id'))

In [202]:
p = curr_nodes.values_list('id',rv_field)
p

<QuerySet [(308271L, 13031L), (308271L, 33789L), (308271L, 28518L), (308271L, 34314L), (308271L, 72063L), (308271L, 70887L), (308271L, 19856L), (308271L, 34289L), (308271L, 506L), (308271L, 27935L), (308271L, 19998L), (308271L, 21354L), (308271L, 30364L), (308271L, 48642L), (308271L, 57186L), (308271L, 50569L), (308271L, 45275L), (308271L, 55746L), (308271L, 55748L), (308271L, 32572L), '...(remaining elements truncated)...']>

In [206]:
%%time
########################################################
###### This is slow but HARD to fix ####################
########################################################
########################################################


reset_database_connection()
hcounts = []
all_nodes = classification.objects.filter(version__name='v4_1_0')
curr_nodes = all_nodes.filter(level__letter = letter)
sids = set(sDB.sequence_set.values_list('id',flat = True))

for node_id in reverse_dict:
    qset = curr_nodes.filter(id = node_id)
    hlist = set( qset.values_list(rv_field, flat =  True) )
    hcount = len( hlist & sids)
    hcounts.append(hcount)
# forward_dict

CPU times: user 5.63 s, sys: 156 ms, total: 5.79 s
Wall time: 51.7 s


In [207]:
%%time
########################################
### New routine for all letters ########
########################################
########################################

# norm_count = dict()
OUTPUT = dict()

l = len(reverse_dict)
OUTPUT = dok_matrix( (l+1,l+1), dtype =  np.float)
# INPUT = D_raw.todok().iteritems()
INPUT = D_raw
it  = using_tocoo_izip( INPUT )



# c=counter(range(6577065),per=100000)
c=counter(it,per=1000)
it  = using_tocoo_izip( INPUT )
h1s = []
h2s = []
h3s = []
for x,y,v in it:
# for (x,y),v in it:
    
    c.count()    
#     h1s += [hcounts[reverse_dict[x]]]
#     h2s += [hcounts[reverse_dict[y]]]
    h1s += [hcounts[x]]
    h2s += [hcounts[y]]
    h3s += [v]

BUFlst = ISS_normalise_new(
    np.array(h1s),
    np.array(h2s),
    np.array(h3s),
)



it  = using_tocoo_izip( INPUT )
d = dict()
for (x,y,_),v in izip( it, BUFlst):
    k = (x,y)
    d[k] = v

# norm_count = OUTPUT
# for x,y,v in using_tocoo_izip(D_raw):
# D_norm = copy(D_raw.todok())

# l = max(hmmids) + 1
# l = len(reverse_dict)
# mat = dok_matrix( shape = (l+1,l+1), dtype =  np.float)
# mat.update(OUTPUT)
OUTPUT.update(d)

# D_norm = dok_matrix( (l+1,l+1),np.float)
# D_norm.update( OUTPUT )

0 of 58969
1000 of 58969
2000 of 58969
3000 of 58969
4000 of 58969
5000 of 58969
6000 of 58969
7000 of 58969
8000 of 58969
9000 of 58969
10000 of 58969
11000 of 58969
12000 of 58969
13000 of 58969
14000 of 58969
15000 of 58969
16000 of 58969
17000 of 58969
18000 of 58969
19000 of 58969
20000 of 58969
21000 of 58969
22000 of 58969
23000 of 58969
24000 of 58969
25000 of 58969
26000 of 58969
27000 of 58969
28000 of 58969
29000 of 58969
30000 of 58969
31000 of 58969
32000 of 58969
33000 of 58969
34000 of 58969
35000 of 58969
36000 of 58969
37000 of 58969
38000 of 58969
39000 of 58969
40000 of 58969
41000 of 58969
42000 of 58969
43000 of 58969
44000 of 58969
45000 of 58969
46000 of 58969
47000 of 58969
48000 of 58969
49000 of 58969
50000 of 58969
51000 of 58969
52000 of 58969
53000 of 58969
54000 of 58969
55000 of 58969
56000 of 58969
57000 of 58969
58000 of 58969
CPU times: user 148 ms, sys: 32 ms, total: 180 ms
Wall time: 143 ms


In [214]:
%%time
import cPickle as pk

fname = 'data/%s_D_norm' % letter
test__norm( OUTPUT, reverse_dict, letter, seqDB_curr = sDB,
          norm_func = ISS_normalise_new)
# test__norm( D_norm, hmms, ISS_normalise_new)
test__key( OUTPUT )
D_norm = OUTPUT

pk.dump(D_norm.tocoo(),open(  fname ,'wb'))
fname = 'data/%s_meta' % letter
pk.dump([reverse_dict,hcounts],open(  fname ,'wb'))

# if 'D_norm' not in locals().keys():
#     D_norm = pk.load(open(fname, 'rb'))
    
# test__norm( D_norm, hmms)

[OK] 1.10.150.240.0 against 4.10.1080.10.0 overlaps: Expected: 0.700187, Actual: 0.700187 from 
0 of -1
[OK] 1.10.730.10.0 against 3.40.50.300.0 overlaps: Expected: 0.373085, Actual: 0.373085 from 
[OK] 3.40.50.1010.0 against 3.40.50.1240.0 overlaps: Expected: 0.282451, Actual: 0.282451 from 
[OK] 1.10.10.10.0 against 2.10.230.10.0 overlaps: Expected: 0.113784, Actual: 0.113784 from 
[OK] 2.60.200.20.0 against 3.20.70.20.0 overlaps: Expected: 0.171979, Actual: 0.171979 from 
[OK]Sorted rate: [ x < y for index (x,y) ) ] is 100.00 %, Expected: 100.00 %
CPU times: user 644 ms, sys: 20 ms, total: 664 ms
Wall time: 3.12 s


In [9]:
%%time
pdata_D_norm = zip(*sorted_D_norm)

CPU times: user 14.6 s, sys: 328 ms, total: 14.9 s
Wall time: 14.9 s


In [120]:
# hcounts_pad = hcounts_
hmmids_pad = hmmids[:]
hmmids_pad.insert(0, None)

hcounts_pad = hcounts[:]
hcounts_pad.insert(0,None)

sfs_pad = sfs[:]
sfs_pad.insert(0,None)

In [144]:
import cPickle as pk
pdata = np.vstack([xss,yss,zss])
fname = 'data/5000pdata'
pk.dump(pdata, open(fname, 'wb'))


In [134]:
import cPickle as pk
pdata = np.vstack([xss,yss,zss])
fname = 'data/1000pdata'
pk.dump(pdata, open(fname, 'wb'))


In [129]:
import cPickle as pk
pdata = np.vstack([xss,yss,zss])
fname = 'data/pdata_raw'
pk.dump(pdata, open(fname, 'wb'))


In [132]:
%%time

D_curr = D_norm
sorted_D_norm = sort_coo(D_curr.tocoo(),order = 1)

it = izip(
# using_tocoo_izip(D_norm),
sorted_D_norm,
sfbools,    
)

lst = sfbools
# c = counter(lst,per = 1)
D_curr = D_raw.todok()

CPU times: user 2.56 s, sys: 128 ms, total: 2.69 s
Wall time: 2.69 s


In [121]:
reset_database_connection()

In [ ]:
# view a HMM-HMM ISS_hit object
# That means we need to construct ISS_hit first

In [165]:
i = 0
c = counter([],INF = 1, per = 10 )
for (hmmid1,hmmid2,v),sfbool in it:
    c.count()   
    if sfbool:
        continue
    
    print v
    raw = D_curr[hmmid1,hmmid2]
    hmmid1 = int(hmmid1)
    hmmid2 = int(hmmid2)
    hmm1 = hmms.get(id = hmmid1 + 1)
    hmm2 = hmms.get(id = hmmid2 + 1)
#     raise
#     if hmm1.cath_node.parent == hmm2.cath_node.parent:
    cnode1 = hmm1.cath_node
    cnode2 = hmm2.cath_node
#     if cnode1.Class == cnode2.Class:
#     if cnode1.topo == cnode2.topo:
    
    if cnode1.homsf == cnode2.homsf:
        continue
    if cnode1.superfamily() in ['3.30.200.20', '1.10.10.530']:
        continue
    
#     if c.i >= 2:
    i+=1
    print hmm1
    print hmm2
    print "norm ISS is %s"%v
    print "raw ISS is %d"%raw
    print hmm1.hits.count() + hmm2.hits.count()
    
    print "http://xhits.cathdb.info/crosshits.php?sf2=%s&sf1=%s&db_source=Crosshits_v4_1_0"%(cnode1.superfamily(),cnode2.superfamily())
    if i == 10:
        break

0 of -1
0.000786053575499
HMM for 3.40.50.720.1.0 
HMM for 3.50.50.60.67.0 
norm ISS is 0.000786053575499
raw ISS is 551
1322
http://xhits.cathdb.info/crosshits.php?sf2=3.40.50.720&sf1=3.50.50.60&db_source=Crosshits_v4_1_0
0.000786053575499
HMM for 3.40.50.720.1.0 
HMM for 3.50.50.60.44.0 
norm ISS is 0.000786053575499
raw ISS is 551
1420
http://xhits.cathdb.info/crosshits.php?sf2=3.40.50.720&sf1=3.50.50.60&db_source=Crosshits_v4_1_0
10 of -1
20 of -1
30 of -1
40 of -1
50 of -1
60 of -1
70 of -1
80 of -1
90 of -1
100 of -1
110 of -1
120 of -1
130 of -1
140 of -1
150 of -1
160 of -1
170 of -1
180 of -1
190 of -1
200 of -1
210 of -1
220 of -1
230 of -1
240 of -1
250 of -1
260 of -1
270 of -1
280 of -1
290 of -1
300 of -1
310 of -1
320 of -1
0.000806989049606
HMM for 1.10.510.10.29.0 
HMM for 3.30.420.40.31.0 
norm ISS is 0.000806989049606
raw ISS is 1612
4098
http://xhits.cathdb.info/crosshits.php?sf2=1.10.510.10&sf1=3.30.420.40&db_source=Crosshits_v4_1_0
0.000806989049606
0.000806989049

HMM for 1.10.510.10.108.0 
HMM for 3.30.420.40.31.0 
norm ISS is 0.0010749868902
raw ISS is 1613
4100
http://xhits.cathdb.info/crosshits.php?sf2=1.10.510.10&sf1=3.30.420.40&db_source=Crosshits_v4_1_0
4930 of -1
0.00107565251744
HMM for 1.10.510.10.55.0 
HMM for 3.30.420.40.31.0 
norm ISS is 0.00107565251744
raw ISS is 1612
4099
http://xhits.cathdb.info/crosshits.php?sf2=1.10.510.10&sf1=3.30.420.40&db_source=Crosshits_v4_1_0


In [16]:
hmmids = list(hmms.values_list('id',flat = True))

In [46]:
key_v = sorted_D_norm[0]
print key_v
# D_temp = D_norm.toarray()

(2775, 19584, -3.2755416884013093)


In [129]:
# hmmid1,hmmid2, v = key_v
# hmmid1 += 1
# hmmid2 += 

hmm1 = HMMprofile.objects.get(id = hmmid1  )
hmm2 = HMMprofile.objects.get(id = hmmid2  )

inter_hitseq = hmm1.hits.all() & hmm2.hits.all()
hits1 = hmm1.hit4hmm2hsp_set.filter(target__in = inter_hitseq)
hits2 = hmm2.hit4hmm2hsp_set.filter(target__in = inter_hitseq)


fmt = "%s,\t domain:%s\t length:%s,\t hitnum:%d"
print "intersection count is",inter_hitseq.count()
print  fmt %(hmm1,hmm1.cath_node.domain, p_hmmlen.findall(hmm1.text)[0], hmm1.hits.count() )
print fmt % (hmm2,hmm2.cath_node.domain, p_hmmlen.findall(hmm2.text)[0], hmm2.hits.count() )
for i,j in zip(hits1,hits2):
    try:
        node =  domain.objects.get(domain_id = i.target.acc).classification
    except:
        node = ''
    print i.target.acc,i.bitscore,
    print j.target.acc,j.bitscore
#     print node
    print i.start,j.start
    print i.end,j.end

intersection count is 0
HMM for 1.20.120.390.1.0 ,	 domain:2icwG01	 length:124,	 hitnum:2
HMM for 2.60.40.10.267.0 ,	 domain:2xy1A02	 length:94,	 hitnum:3480


In [86]:
hits1_ids = hmm1.hits.values_list('id',flat = 1)
hits2_ids = hmm2.hits.values_list('id',flat = 1)
inter_hitseq = set( hits1_ids ) & set( hits2_ids )

hits1 = list(hmm1.hit4hmm2hsp_set.filter(target__id__in = inter_hitseq))
hits2 = list(hmm2.hit4hmm2hsp_set.filter(target__id__in = inter_hitseq))

fmt = "%s,\t domain:%s\t length:%s,\t hitnum:%d"
for i,j in zip(hits1,hits2):
    print i.target.acc,i.bitscore,
    print j.target.acc,j.bitscore
#     print node
    print i.start,j.start
    print i.end,j.end

1gn0A00 73.6 1gn0A00 12.4
0 7
105 96


In [222]:
type('stt')

str

In [ ]:
def compare_hits( hmm1,hmm2):
#     for varn in ['hmm1','hmm2']:
#         var = eval(varn)
#         if 
#             exec("%s=classification.objects.get_bytree("")")
#     if type()=='str':
#         hmm
    hits1_ids = hmm1.hits.values_list('id')
    hits2_ids = hmm2.hits.values_list('id')
    inter_hitseq = set( hits1_ids ) & set( hits2_ids )
    hits1 = hmm1.hit4hmm2hsp_set.prefetch_related('target')
    hits2 = hmm2.hit4hmm2hsp_set.prefetch_related('target')
#     hits2 = hmm2.hit4hmm2hsp_set.filter(target__in = inter_hitseq)
    fmt = "%s,\t domain:%s\t length:%s,\t hitnum:%d"
#     print "intersection count is",inter_hitseq.count()
    print "intersection count is",inter_hitseq.__len__()
    print  fmt %(hmm1,hmm1.cath_node.domain, p_hmmlen.findall(hmm1.text)[0], hmm1.hits.count() )
    print  fmt % (hmm2,hmm2.cath_node.domain, p_hmmlen.findall(hmm2.text)[0], hmm2.hits.count() )
#     for i,j in zip(hits1,hits2):
    for hitid in inter_hitseq:
        i = hits1.get(target=hitid)
        j = hits2.get(target=hitid)
        try:
            node =  domain.objects.get(domain_id = i.target.acc).classification
        except Exception as e:
#             print e
            node = ''
        print i.target.acc,i.bitscore,
        print j.target.acc,j.bitscore
        if node: print node
        
        print i.start,j.start
        print i.end,j.end
        
        
        

In [245]:
d1 = dict(zip(levels,lst))

d1.pop(None)
d1.pop('root')
# d1

30

In [229]:
node_str="3.30.40.10.2.0"
lst = [int(x) for x in node_str.split('.')]

In [261]:
node_str="3.30.40.10.2.0"
lst = [int(x) for x in node_str.split('.')]
d1 = dict(zip(levels[2:],lst))
node_str="3.30.60.20.17.0"
lst = [int(x) for x in node_str.split('.')]
d2 = dict(zip(levels[2:],lst))
# d2.pop(None)
# d2.pop('root')
node1 = classification.objects.get(**d1)
node2 = classification.objects.get(**d2)

# node1 = all_nodes.get_bytree("3.30.40.10.2.0")[0]
# node2 = all_nodes.get_bytree("3.30.60.20.17.0")[0]
hmm1 = node1.hmmprofile
hmm2 = node2.hmmprofile
compare_hits(hmm1,hmm2)
# hits1_ids = hmm1.hits.values_list('id')
# hits2_ids = hmm2.hits.values_list('id')

intersection count is 97
HMM for 3.30.40.10.2.0 ,	 domain:3t7lA00	 length:90,	 hitnum:146
HMM for 3.30.60.20.17.0 ,	 domain:1rfhA01	 length:46,	 hitnum:128
2en2A01 20.2 2en2A01 16.4
3.30.160.60.33.0
0 0
32 32
2emlA01 30.5 2emlA01 22.6
0 0
32 21
2yuuA00 14.1 2yuuA00 34.4
6 22
77 61
2wbuA03 18.8 2wbuA03 15.2
3.30.160.60.24.0
0 0
27 26
2yqmA01 64.6 2yqmA01 9.6
0 0
58 34
2m9aA02 14.8 2m9aA02 15.4
0 0
27 17
1a1hA03 23.6 1a1hA03 17.8
3.30.160.60.5.0
0 0
27 26
2em7A01 23.6 2em7A01 20.4
0 0
32 30
3cxlA02 11.1 3cxlA02 41.3
3.30.60.20.11.0
1 12
73 53
3uffA01 13.2 3uffA01 46.9
2 6
54 47
1meyC02 33.1 1meyC02 26.5
0 0
26 26
2eomA01 27.5 2eomA01 12.1
0 0
32 21
2i13A04 31.6 2i13A04 24.1
0 0
28 21
2em8A01 21.4 2em8A01 17.6
0 0
31 20
3ueyA01 12.9 3ueyA01 46.7
3 6
54 47
1x62A01 13.5 1x62A01 13.9
1 1
53 43
2i13A05 31.1 2i13A05 23.9
0 0
31 21
2en3A01 25.2 2en3A01 19.8
0 0
33 18
1jocA02 74.7 1jocA02 6.0
0 5
63 60
2jp9A03 25.2 2jp9A03 8.7
0 0
28 28
1a1iA01 16.7 1a1iA01 9.9
0 0
32 32
1meyC03 35.9 1meyC03 33.

In [264]:
hitlist1 = hmm1.hit4hmm2hsp_set
hitlist2 = hmm2.hit4hmm2hsp_set


In [265]:
	hits1_ids = hitlist1.values_list('target',flat = True)
	hits2_ids = hitlist2.values_list('target',flat = True)
	inter_ids = set( hits1_ids ) & set( hits2_ids )


In [269]:
	qset = zip( 
		hitlist1.filter(target__in = inter_ids).order_by("target"),
		hitlist2.filter(target__in = inter_ids).order_by("target"),
		)


[(<hit4hmm2hsp: Query:HMM for 3.30.40.10.2.0  
  Target:1a1hA01 from <class 'tst.models.seqDB'>>,
  <hit4hmm2hsp: Query:HMM for 3.30.60.20.17.0  
  Target:1a1hA01 from <class 'tst.models.seqDB'>>),
 (<hit4hmm2hsp: Query:HMM for 3.30.40.10.2.0  
  Target:1a1hA03 from <class 'tst.models.seqDB'>>,
  <hit4hmm2hsp: Query:HMM for 3.30.60.20.17.0  
  Target:1a1hA03 from <class 'tst.models.seqDB'>>),
 (<hit4hmm2hsp: Query:HMM for 3.30.40.10.2.0  
  Target:1a1iA01 from <class 'tst.models.seqDB'>>,
  <hit4hmm2hsp: Query:HMM for 3.30.60.20.17.0  
  Target:1a1iA01 from <class 'tst.models.seqDB'>>),
 (<hit4hmm2hsp: Query:HMM for 3.30.40.10.2.0  
  Target:1aayA01 from <class 'tst.models.seqDB'>>,
  <hit4hmm2hsp: Query:HMM for 3.30.60.20.17.0  
  Target:1aayA01 from <class 'tst.models.seqDB'>>),
 (<hit4hmm2hsp: Query:HMM for 3.30.40.10.2.0  
  Target:1f2iG02 from <class 'tst.models.seqDB'>>,
  <hit4hmm2hsp: Query:HMM for 3.30.60.20.17.0  
  Target:1f2iG02 from <class 'tst.models.seqDB'>>),
 (<hit4hmm